In [1]:
import numpy as np
import json
from random import shuffle

In [2]:
in_filename = "instances/huge.json"

In [3]:
with open(in_filename, 'rb') as f:
    inst = json.load(f)

In [4]:
def parser_jobs(jobs):
    parsed = []
    for job in jobs:
        j  = job['job']
        Sj = job['sequence']
        rj = job['release_date']
        dj = job['due_date']
        wj = job['weight']
        parsed.append((j, Sj, rj, dj, wj))
    return parsed

def parser_tasks(tasks):
    parsed = []
    for task in tasks:
        i = task['task']
        pi = task['processing_time']
        Mi = parser_machines(task['machines'])
        parsed.append((i, pi, Mi))
    return parsed

def parser_machines(machines):
    parsed = []
    for machine in machines:
        m = machine['machine']
        Oim = machine['operators']
        parsed.append((m, Oim))
    return parsed

def parser_inst(inst):
    J = inst['parameters']['size']['nb_jobs']
    I = inst['parameters']['size']['nb_tasks']
    M = inst['parameters']['size']['nb_machines']
    O = inst['parameters']['size']['nb_operators']
    alpha = inst['parameters']['costs']['unit_penalty']
    beta = inst['parameters']['costs']['tardiness']
    jobs = parser_jobs(inst['jobs'])
    tasks = parser_tasks(inst['tasks'])
    return (J, I, M, O, alpha, beta, jobs, tasks)

In [5]:
(J, I, M, O, alpha, beta, jobs, tasks) = parser_inst(inst)

In [6]:
print(f"J={J}\nI={I}\nM={M}\nO={O}\nα={alpha}\nβ={beta}")

J=200
I=1000
M=50
O=30
α=6
β=1


In [7]:
def job_weight(job):
    dj, wj = job[3], job[4]
    Cj = T # global variable, proxy for job's time of completion
    Tj = max(Cj-dj, 0)
    Uj = 1 if Tj > 0 else 0
    return wj*(Cj + alpha*Uj + beta*Tj)

In [8]:
def job_weight2(job):
    dj, wj = job[3], job[4]
    s = 0
    for future_task_id in np.setdiff1d(job[1], [_[0] for _ in done]):
        s += tasks[future_task_id-1][1]
    Cj = T + s # proxy for job's time of completion
    Tj = max(Cj-dj, 0)
    Uj = 1 if Tj > 0 else 0
    return wj*(Cj + alpha*Uj + beta*Tj)

In [9]:
def task_weight(task):
    job = jobs[tid2jid(task[0])-1]
    dj, wj = job[3], job[4]
    t_idx = job[1].index(task[0]) # index of task in all job's tasks
    s = 0
    for future_task_id in job[1][t_idx:]:
        s += tasks[future_task_id-1][1]
    Cj = T + s # proxy for job's time of completion
    Tj = max(Cj-dj, 0)
    Uj = 1 if Tj > 0 else 0
    return wj*(Cj + alpha*Uj + beta*Tj)

In [10]:
def tid2jid(tid):
    for job in jobs:
        if tid in job[1]:
            return job[0]

In [11]:
def cost(done):
    C = {}
    for job in jobs:
        C[job[0]] = 0
    for done_task in done:
        jid = tid2jid(done_task[0])
        for job_task_id in jobs[jid-1][1]:
            if job_task_id == done_task[0]:
                t = tasks[job_task_id-1]
        C[jid] = max(C[jid], done_task[1]+t[1]) # end time for task
    s = 0
    for job in jobs:
        dj, wj = job[3], job[4]
        Cj = C[job[0]] # global variable, proxy for job's time of completion
        Tj = max(Cj-dj, 0)
        Uj = 1 if Tj > 0 else 0
        s += wj*(Cj + alpha*Uj + beta*Tj)
    return s

In [12]:
def optim():
    running = []
    done = []
    busy_operators = []
    busy_machines = []

    T = 0
    while len(done) < I:

        """
        feasible_tasks = []
        feasible_tasks_id = []
        for j in jobs:
            if T >= j[2]:
                for task_id in j[1]:
                    if (not task_id in [_[0] for _ in running]) \
                    and (not task_id in [_[0] for _ in done]):
                        feasible_tasks.append(tasks[task_id-1])
                        feasible_tasks_id.append(task_id)

        stasks = sorted(feasible_tasks, key=task_weight)
        #stasks = sorted(feasible_tasks, key=lambda _: _[1])

        for t in stasks:
            task_id = t[0]
            j = jobs[tid2jid(task_id)-1]
            t_idx = j[1].index(task_id)
            for m in t[2]: # look for a machine to start the task on
                m_id = m[0] # get this machine's id
                for op_id in m[1]: # look for an operator to operate machine
                    if (not op_id in busy_operators) \
                    and (not m_id in busy_machines) \
                    and (not task_id in [_[0] for _ in running]) \
                    and (not task_id in [_[0] for _ in done]) \
                    and np.intersect1d(j[1][:t_idx], [_[0] for _ in running]).size == 0 \
                    and np.isin(j[1][:t_idx], [_[0] for _ in done]).all(): # all prior tasks done
                        running.append((task_id, T, m_id, op_id))
                        busy_operators.append(op_id)
                        busy_machines.append(m_id)
        """

        ##########################


        #sjobs = sorted(jobs, key=job_weight2)
        #sjobs = list(reversed(sorted(jobs, key=job_weight)))
        for j in jobs:
            if T >= j[2]: # it is past the release date of this job
                for (t_idx, task_id) in enumerate(j[1]): # loop over tasks for this job
                    t = tasks[task_id-1] # get full task for this task id
                    for m in t[2]: # look for a machine to start the task on
                        m_id = m[0] # get this machine's id
                        for op_id in m[1]: # look for an operator to operate machine
                            if (not op_id in busy_operators) \
                            and (not m_id in busy_machines) \
                            and (not task_id in [_[0] for _ in running]) \
                            and (not task_id in [_[0] for _ in done]) \
                            and np.intersect1d(j[1][:t_idx], [_[0] for _ in running]).size == 0 \
                            and np.isin(j[1][:t_idx], [_[0] for _ in done]).all(): # all prior tasks done
                                running.append((task_id, T, m_id, op_id))
                                busy_operators.append(op_id)
                                busy_machines.append(m_id)

        for running_task in running:
            t_id, start_time, m_id, op_id = running_task
            processing_time = tasks[t_id-1][1]
            if (T - start_time >= processing_time):
                running = [_ for _ in running if _[0] != t_id]
                busy_operators.remove(op_id)
                busy_machines.remove(m_id)
                done.append((t_id, start_time, m_id, op_id))

        T += 1 # time flows
    
    return done

In [13]:
done = optim()

In [14]:
cost(done)

96063

In [15]:
def jsonify(done):
    res = []
    for task in done:
        sub_res = {}
        sub_res['task'], sub_res['start'], sub_res['machine'], sub_res['operator'] = task
        res.append(sub_res)
    return res

In [16]:
out_filename = "solutions/KIRO-huge.json"
with open(out_filename, 'w') as f:
    json.dump(jsonify(best_done), f)

NameError: name 'best_done' is not defined